<a href="https://colab.research.google.com/github/skorpiom/traffic_sign_rec/blob/master/traffic_sign_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!rm -rf sample_data

In [0]:
%tensorflow_version 2.x
import os
import csv
import numpy as np
import pandas as pd
from pandas.io.parsers import read_csv
import matplotlib.pyplot as plt
import shutil
import plotly.graph_objects as go
from sklearn.metrics import confusion_matrix, classification_report

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.applications import VGG16

np.set_printoptions(precision = 6, suppress = True)

In [0]:
#download training data, test data (signs in random order) and legend
!wget https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB-Training_fixed.zip
!wget https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Test_Images.zip
!wget https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Test_GT.zip

#unzip
!unzip -q GTSRB-Training_fixed.zip 
!unzip -q GTSRB_Final_Test_Images.zip
!unzip -q GTSRB_Final_Test_GT.zip

#delete downloaded zip archives
!rm -rf ./GTSRB-Training_fixed.zip
!rm -rf ./GTSRB_Final_Test_GT.zip
!rm -rf ./GTSRB_Final_Test_Images.zip
!rm -rf ./GTSRB/Final_Test/Images/GT-final_test.test.csv

In [0]:
base_dir = './GTSRB/Training'
raw_no_of_files = {}
classes = []

#list raw sub-category names
for n in range(0,43):
  if n<10:
    classes.append("0000"+str(n))
  else:
    classes.append("000"+str(n))

for dir in classes:
  raw_no_of_files[dir] = len(os.listdir(os.path.join(base_dir,dir)))

In [0]:
#organizing test, valid and test directory

data_dir = './train_and_valid_signs'
dirs_train = []
dirs_valid = []
dirs_test = []

train_dir = os.path.join(data_dir,'train') #training catalog
valid_dir = os.path.join(data_dir,'valid') #valid catalog
test_dir = os.path.join(data_dir,'test') #tets catalog

for n in range(0,43):
  dirs_train.append(os.path.join(train_dir,classes[n])) #finall training signs catalogs 

for n in range(0,43):
  dirs_valid.append(os.path.join(valid_dir,classes[n])) #finall validation signs catalogs

for n in range(0,43):
  dirs_test.append(os.path.join(test_dir,classes[n])) #finall testing signs catalogs

if not os.path.exists(data_dir):
  os.mkdir(data_dir)
  
#creating main catalogs (training, validation, testing)
for directory in (train_dir,valid_dir,test_dir):
  if not os.path.exists(directory):
    os.mkdir(directory)

#creating subcatalogs for each main path (signs category "00000","00001"..."00042")
for directory in dirs_train:
  if not os.path.exists(directory):
    os.mkdir(directory)

for directory in dirs_valid:
  if not os.path.exists(directory):
    os.mkdir(directory)

for directory in dirs_test:
  if not os.path.exists(directory):
    os.mkdir(directory)

#training and validation data!
#copy signs from source to destinantion directory
#set is divided for training data 90% and validation data 10%

for i,fname in enumerate(sign_fnames):
  for j,name in enumerate(fname):
    if j<=0.9*int(np.floor(len(fname))):
      src = os.path.join(base_dir,classes[i],name)
      dst = os.path.join(dirs_train[i],name)
      shutil.copyfile(src,dst)
    if 0.9*int(np.floor(len(fname))) < j <= len(fname):
      src = os.path.join(base_dir,classes[i],name)
      dst = os.path.join(dirs_valid[i],name)
      shutil.copyfile(src,dst)

#testing data!
#creating map based on csv file (contain files names and classes) 
#key = sign category, value = sign name
signs_classes = {}
file_name = ""
with open('./GT-final_test.csv', mode='r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=';')
    line_count = 0
    for row in csv_reader:
      line_count += 1
      if 1 < line_count:
        signs_classes[row[0]] = row[7]

#copy data from source to destination directory

signs_test_names = []
signs_test_names.append(os.listdir('./GTSRB/Final_Test/Images'))

for j,name in enumerate(signs_test_names):
  for k,n in enumerate(name):
    src = os.path.join('./GTSRB/Final_Test/Images',n)
    dst = os.path.join(dirs_test[int(signs_classes[n])],n)
    shutil.copyfile(src,dst)

In [0]:
#display number of signs in each class
sign_fnames = []
for ind in range(0,43):
  sign_fnames.append(os.listdir(os.path.join(base_dir,classes[ind])))  

for i in range(0,43):
  print(i,' liczba obiektow: ',len(sign_fnames[i]))

In [0]:
#display number of signs in each training and validation class

for i in range(0,43):
  print(i,' zbior treningowy ',len(os.listdir(dirs_train[i])))
  print(i,' zbior walidacyjny ',len(os.listdir(dirs_valid[i])))

#DISPLAY INDICATED SIGN

In [0]:
img_path = os.path.join('./train_and_valid_signs','test','00039','02050.ppm')
print(img_path)
img = image.load_img(img_path)
plt.figure(figsize=(1,1))
plt.imshow(img)
plt.grid(False)
plt.axis(False)

#AUGMENTATION

In [0]:
#data augmentation

train_datagen = ImageDataGenerator(
    rotation_range = 40,
    rescale = 1./255.,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

valid_datagen = ImageDataGenerator(rescale = 1./255.)                           #data scaling

train_generator = train_datagen.flow_from_directory(directory = train_dir,
                                                    target_size=(32,32),        #signs dimensions
                                                    batch_size = 16,            #size of batch
                                                    class_mode='categorical')   #43 classes of signs exists
valid_generator = train_datagen.flow_from_directory(directory = valid_dir,
                                                    target_size=(32,32),
                                                    batch_size = 16,
                                                    class_mode='categorical')

#TRANSFER LEARNING //not used

In [0]:
conv_base = VGG16(weights = 'imagenet', include_top = False, input_shape=(32,32,3))
conv_base.trainable = False

def print_layers(model):
  for layer in model.layers:
    print(f'layer_name {layer.name:13} trainable: {layer.trainable}')

set_trainable = False
for layer in conv_base.layers:
  if layer.name =='block5_conv1':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

print_layers(conv_base)


In [0]:
model = Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(units = 1024, activation = 'relu'))
model.add(layers.Dense(units = 43, activation = 'softmax'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_5 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1024)              525312    
_________________________________________________________________
dense_11 (Dense)             (None, 43)                44075     
Total params: 15,284,075
Trainable params: 569,387
Non-trainable params: 14,714,688
_________________________________________________________________


In [0]:
model.compile(optimizer=optimizers.RMSprop(lr=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
history = model.fit_generator(generator=train_generator,
                              epochs = 50,
                              validation_data = valid_generator,
                              callbacks = [tensorboard])

#MODEL

In [0]:
model = Sequential()
model.add(layers.Conv2D(filters=32, kernel_size=(3,3),activation='relu',input_shape=(32, 32, 3)))
model.add(layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Conv2D(filters=128,kernel_size=(3,3),activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Conv2D(filters=256,kernel_size=(3,3),activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(units=2048, activation='relu'))
model.add(layers.Dense(units=43, activation='softmax'))
model.summary()

#TRAINING

In [0]:
#training with RMSprop optimizer
model.compile(optimizer = optimizers.RMSprop(lr=1e-4), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
!rm -rf logs
!mkdir logs
tensorboard = TensorBoard(log_dir='logs')

In [0]:
history = model.fit_generator(generator=train_generator,
                              epochs = 50,
                              validation_data = valid_generator,
                              callbacks = [tensorboard])

In [0]:
#training with adadelta optimizer
model.compile(optimizer = 'adadelta', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
!rm -rf logs
!mkdir logs
tensorboard = TensorBoard(log_dir='logs')

In [0]:
history = model.fit_generator(generator=train_generator,
                              epochs = 50,
                              validation_data = valid_generator,
                              callbacks = [tensorboard])

In [0]:
#charts created by tensorflow

%load_ext tensorboard
!tensorboard dev upload --logdir logs

#TEST SET

In [0]:
#testing model with new data
test_datagen = ImageDataGenerator(rescale=1./255.)
test_generator = test_datagen.flow_from_directory(
    './train_and_valid_signs/test/',
    target_size = (32,32),
    batch_size = 1,
    class_mode = 'categorical',
    shuffle = False
)

y_prob = model.predict_generator(test_generator, test_generator.samples)

In [0]:
#creating an array of predicted answers (category)
y_pred = np.argmax(y_prob,axis=1)
y_pred

In [0]:
#display predictions
predictions = pd.DataFrame({'class':y_pred})
predictions

In [0]:
#creating an array of true answers (category)
y_true = test_generator.classes
y_true

In [0]:
#creating a confusion matrix
cm = confusion_matrix(y_true,y_pred)
print(cm[39])

In [0]:
#print raport
print(classification_report(y_true,y_pred,target_names=test_generator.class_indices.keys()))